## Setting Up:

In [ ]:
import sys
import os

import logging
import warnings
from datetime import datetime
from collections import defaultdict, Counter
from functools import partial

import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt
from matplotlib.patches import Patch
from matplotlib.lines import Line2D
from tqdm.notebook import tqdm
from cmcrameri import cm
from skorch.helper import SliceDataset
from skorch.callbacks import EarlyStopping, LRScheduler, Checkpoint

import massbalancemachine as mbm

# Add root of repo to import MBM
sys.path.append(os.path.join(os.getcwd(), '../../'))

# Local modules
from scripts.helpers import *
from scripts.glamos_preprocess import *
from scripts.plots import *
from scripts.config_CH import *
from scripts.nn_helpers import *
from scripts.xgb_helpers import *
from scripts.geodata import *
from scripts.NN_networks import *
from scripts.geodata_plots import *

warnings.filterwarnings('ignore')

%load_ext autoreload
%autoreload 2

cfg = mbm.SwitzerlandConfig()

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, Subset
from torch.utils.data import WeightedRandomSampler, SubsetRandomSampler
from torch.optim.lr_scheduler import ReduceLROnPlateau

seed_all(cfg.seed)
print("Using seed:", cfg.seed)

if torch.cuda.is_available():
    print("CUDA is available")
    free_up_cuda()
else:
    print("CUDA is NOT available")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [ ]:
# Plot styles:
path_style_sheet = 'scripts/example.mplstyle'
plt.style.use(path_style_sheet)

colors = get_cmap_hex(cm.batlow, 10)
color_annual = "#c51b7d"
color_winter = colors[0]

vois_climate = [
    't2m',
    'tp',
    'slhf',
    'sshf',
    'ssrd',
    'fal',
    'str',
]

vois_topographical = ["aspect_sgi", "slope_sgi", "svf"]

# Read GLAMOS stake data
data_glamos = getStakesData(cfg)

# Compute padding for monthly data
months_head_pad, months_tail_pad = mbm.data_processing.utils._compute_head_tail_pads_from_df(
    data_glamos)

# Initialize logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(message)s')

# Transform data to monthly format (run or load data):
paths = {
    'csv_path': cfg.dataPath + path_PMB_GLAMOS_csv,
    'era5_climate_data':
    cfg.dataPath + path_ERA5_raw + 'era5_monthly_averaged_data.nc',
    'geopotential_data':
    cfg.dataPath + path_ERA5_raw + 'era5_geopotential_pressure.nc',
    'radiation_save_path': cfg.dataPath + path_pcsr + 'zarr/'
}
RUN = False
data_monthly = process_or_load_data(
    run_flag=RUN,
    data_glamos=data_glamos,
    paths=paths,
    cfg=cfg,
    vois_climate=vois_climate,
    vois_topographical=vois_topographical,
    output_file='CH_wgms_dataset_monthly_LSTM_gs_no_oggm.csv')

# Create DataLoader
dataloader_gl = mbm.dataloader.DataLoader(cfg,
                                          data=data_monthly,
                                          random_seed=cfg.seed,
                                          meta_data_columns=cfg.metaData)

## Input data:

In [ ]:
# Ensure all test glaciers exist in the dataset
existing_glaciers = set(data_monthly.GLACIER.unique())
missing_glaciers = [g for g in TEST_GLACIERS if g not in existing_glaciers]

# Define training glaciers correctly
train_glaciers = [i for i in existing_glaciers if i not in TEST_GLACIERS]

data_test = data_monthly[data_monthly.GLACIER.isin(TEST_GLACIERS)]
data_train = data_monthly[data_monthly.GLACIER.isin(train_glaciers)]
splits, test_set, train_set = get_CV_splits(dataloader_gl,
                                            test_split_on='GLACIER',
                                            test_splits=TEST_GLACIERS,
                                            random_state=cfg.seed)
# Validation and train split:
data_train = train_set['df_X']
data_train['y'] = train_set['y']
data_test = test_set['df_X']
data_test['y'] = test_set['y']

### Feature distribution of test set:

In [ ]:
MONTHLY_COLS = [
    't2m',
    'tp',
    'slhf',
    'sshf',
    'ssrd',
    'fal',
    'str',
    'pcsr',
    'ELEVATION_DIFFERENCE',
]
STATIC_COLS = ['aspect_sgi', 'slope_sgi', 'svf']

feature_columns = MONTHLY_COLS + STATIC_COLS
cfg.setFeatures(feature_columns)

In [ ]:
colors = get_cmap_hex(cm.batlow, 10)
color_dark_blue = colors[0]
custom_palette = {'Train': color_dark_blue, 'Test': '#b2182b'}

fig = plot_tsne_overlap(data_train,
                        data_test,
                        STATIC_COLS,
                        MONTHLY_COLS,
                        sublabels=("a", "b", "c"),
                        label_fmt="({})",
                        label_xy=(0.02, 0.98),
                        label_fontsize=14,
                        n_iter=1000,
                        random_state=cfg.seed,
                        custom_palette=custom_palette)
# save figure
fig.savefig('figures/paper/fig_tsne_overlap_train_test_CH.png',
            dpi=300,
            bbox_inches='tight')

## LSTM:

### Build LSTM dataloaders:

In [ ]:
seed_all(cfg.seed)

df_train = data_train.copy()
df_train['PERIOD'] = df_train['PERIOD'].str.strip().str.lower()

df_test = data_test.copy()
df_test['PERIOD'] = df_test['PERIOD'].str.strip().str.lower()

# --- build train dataset from dataframe ---
ds_train = mbm.data_processing.MBSequenceDataset.from_dataframe(
    df_train,
    MONTHLY_COLS,
    STATIC_COLS,
    months_tail_pad=months_tail_pad,
    months_head_pad=months_head_pad,
    expect_target=True,
    normalize_target=True)

ds_test = mbm.data_processing.MBSequenceDataset.from_dataframe(
    df_test,
    MONTHLY_COLS,
    STATIC_COLS,
    months_tail_pad=months_tail_pad,
    months_head_pad=months_head_pad,
    expect_target=True,
    normalize_target=True)

train_idx, val_idx = mbm.data_processing.MBSequenceDataset.split_indices(
    len(ds_train), val_ratio=0.2, seed=cfg.seed)

### Define & train model:

In [ ]:
custom_params = {
    'lr': 0.001,
    'weight_decay': 0.0001,
    'hidden_size': 128,
    'num_layers': 2,
    'dropout': 0.2,
    'head_dropout': 0.0,
    'static_layers': 2,
    'static_hidden': [128, 64],
    'static_dropout': 0.1,
    'Fm': 9,
    'Fs': 3,
    'bidirectional': False,
    'loss_name': 'neutral',
    'loss_spec': None,
    'two_heads': False
}

# --- build model, resolve loss, train, reload best ---
model_filename = f"models/lstm_model_2025-11-07_no_oggm_norm_y_.pt"

# --- loaders (fit scalers on TRAIN, apply to whole ds_train) ---
seed_all(cfg.seed)
ds_train_copy = mbm.data_processing.MBSequenceDataset._clone_untransformed_dataset(
    ds_train)
ds_test_copy = mbm.data_processing.MBSequenceDataset._clone_untransformed_dataset(
    ds_test)

train_dl, val_dl = ds_train_copy.make_loaders(
    train_idx=train_idx,
    val_idx=val_idx,
    batch_size_train=64,
    batch_size_val=128,
    seed=cfg.seed,
    fit_and_transform=
    True,  # fit scalers on TRAIN and transform Xm/Xs/y in-place
    shuffle_train=True,
    use_weighted_sampler=True  # use weighted sampler for training
)

# --- test loader (copies TRAIN scalers into ds_test and transforms it) ---
test_dl = mbm.data_processing.MBSequenceDataset.make_test_loader(
    ds_test_copy, ds_train_copy, batch_size=128, seed=cfg.seed)

# --- build model, resolve loss, train, reload best ---
seed_all(cfg.seed)
model = mbm.models.LSTM_MB.build_model_from_params(cfg, custom_params, device)
loss_fn = mbm.models.LSTM_MB.resolve_loss_fn(custom_params)

# Evaluate on test
state = torch.load(model_filename, map_location=device)
model.load_state_dict(state)
test_metrics, test_df_preds = model.evaluate_with_preds(
    device, test_dl, ds_test_copy)
test_rmse_a, test_rmse_w = test_metrics['RMSE_annual'], test_metrics[
    'RMSE_winter']

print('Test RMSE annual: {:.3f} | winter: {:.3f}'.format(
    test_rmse_a, test_rmse_w))

## Validation OOS:
Out of sample on test set

In [ ]:
colors = get_cmap_hex(cm.batlow, 10)
color_annual = "#c51b7d"
color_winter = colors[0]

# Areas (with clariden alias fix)
gl_area = get_gl_area(cfg)
gl_area["clariden"] = gl_area["claridenL"]

In [ ]:
scores_annual, scores_winter = compute_seasonal_scores(test_df_preds,
                                                       target_col='target',
                                                       pred_col='pred')

print("Annual scores:", scores_annual)
print("Winter scores:", scores_winter)

fig = plot_predictions_summary(
    grouped_ids=test_df_preds,
    scores_annual=scores_annual,
    scores_winter=scores_winter,
    ax_xlim=(-8, 6),
    ax_ylim=(-8, 6),
    color_annual=color_annual,
    color_winter=color_winter,
)
# save figure
fig.savefig('figures/paper/fig_predvsobs.png', dpi=300, bbox_inches='tight')

In [ ]:
gl_per_el = data_glamos[data_glamos.PERIOD == 'annual'].groupby(
    ['GLACIER'])['POINT_ELEVATION'].mean()
gl_per_el = gl_per_el.sort_values(ascending=False)

test_gl_per_el = gl_per_el[TEST_GLACIERS].sort_values().index

fig, axs = plt.subplots(2, 4, figsize=(32, 15), sharex=True)

gl_per_el = data_glamos[data_glamos.PERIOD == 'annual'].groupby(
    ['GLACIER'])['POINT_ELEVATION'].mean()
gl_per_el = gl_per_el.sort_values(ascending=False)
test_df_preds['gl_elv'] = test_df_preds['GLACIER'].map(gl_per_el)

subplot_labels = [
    '(a)', '(b)', '(c)', '(d)', '(e)', '(f)', '(g)', '(h)', '(i)'
]

axs = PlotIndividualGlacierPredVsTruth(test_df_preds,
                                       axs=axs,
                                       subplot_labels=subplot_labels,
                                       color_annual=color_annual,
                                       color_winter=color_winter,
                                       custom_order=test_gl_per_el,
                                       gl_area=gl_area)

axs[3].set_ylabel("Modeled PMB [m w.e.]", fontsize=20)

fig.supxlabel('Observed PMB [m w.e.]', fontsize=20, y=0.06)
# two distinct handles
legend_scatter_annual = Line2D([0], [0],
                               marker='o',
                               linestyle='None',
                               linewidth=0,
                               markersize=10,
                               markerfacecolor=color_annual,
                               markeredgecolor='k',
                               markeredgewidth=0.8,
                               label='Annual')

legend_scatter_winter = Line2D([0], [0],
                               marker='o',
                               linestyle='None',
                               linewidth=0,
                               markersize=10,
                               markerfacecolor=color_winter,
                               markeredgecolor='k',
                               markeredgewidth=0.8,
                               label='Winter')

# if you already have other handles (e.g., bands/means), append these:
# handles = existing_handles + [legend_scatter_annual, legend_scatter_winter]
handles = [legend_scatter_annual, legend_scatter_winter]

# You can let matplotlib use the labels from the handles; no need to pass `labels=...`
fig.legend(handles=handles,
           loc='upper center',
           bbox_to_anchor=(0.5, 0.05),
           ncol=4,
           fontsize=20)

plt.subplots_adjust(hspace=0.25, wspace=0.25)
plt.show()

# save figure
fig.savefig('figures/paper/fig_predvsobs_indv.png',
            dpi=300,
            bbox_inches='tight')

## Intermediate validation OOS and IS:

In [ ]:
# Geodetic MB + per-glacier periods
geodetic_mb = get_geodetic_MB(cfg)
periods_per_glacier, geoMB_per_glacier = build_periods_per_glacier(geodetic_mb)

In [ ]:
PATH_PREDICTIONS_LSTM_OOS = os.path.join(
    cfg.dataPath, "GLAMOS", "distributed_MB_grids",
    "MBM/testing_LSTM/LSTM_no_oggm_OOS_norm_y")

PATH_PREDICTIONS_LSTM_IS = os.path.join(
    cfg.dataPath, "GLAMOS", "distributed_MB_grids",
    "MBM/testing_LSTM/LSTM_no_oggm_IS_norm_y")

In [ ]:
# Available glaciers (those with LSTM predictions)
glaciers_in_glamos = set(os.listdir(PATH_PREDICTIONS_LSTM_OOS))

# Areas (with clariden alias fix)
gl_area = get_gl_area(cfg)
gl_area["clariden"] = gl_area["claridenL"]

# Glaciers present in both geodetic periods and predictions, sorted by area (asc)
glacier_list = sorted(
    (g for g in periods_per_glacier.keys() if g in glaciers_in_glamos),
    key=lambda g: gl_area.get(g, 0))
print("Number of glaciers:", len(glacier_list))
print("Glaciers:", glacier_list)

In [ ]:
# Run comparison
ds_lstm_OS = process_geodetic_mass_balance_comparison(
    glacier_list=glacier_list,
    path_SMB_GLAMOS_csv=os.path.join(cfg.dataPath, path_SMB_GLAMOS_csv),
    periods_per_glacier=periods_per_glacier,
    geoMB_per_glacier=geoMB_per_glacier,
    gl_area=gl_area,
    test_glaciers=TEST_GLACIERS,
    path_predictions=PATH_PREDICTIONS_LSTM_OOS,
    cfg=cfg,
)

# Drop rows where any required columns are NaN
ds_lstm_OS = ds_lstm_OS.dropna(subset=['Geodetic MB', 'MBM MB'])
ds_lstm_OS = ds_lstm_OS.sort_values(by='Area')
ds_lstm_OS['GLACIER'] = ds_lstm_OS['GLACIER'].apply(lambda x: x.capitalize())

# Run comparison
ds_lstm_IS = process_geodetic_mass_balance_comparison(
    glacier_list=glacier_list,
    path_SMB_GLAMOS_csv=os.path.join(cfg.dataPath, path_SMB_GLAMOS_csv),
    periods_per_glacier=periods_per_glacier,
    geoMB_per_glacier=geoMB_per_glacier,
    gl_area=gl_area,
    test_glaciers=TEST_GLACIERS,
    path_predictions=PATH_PREDICTIONS_LSTM_IS,
    cfg=cfg,
)

# Drop rows where any required columns are NaN
ds_lstm_IS = ds_lstm_IS.dropna(subset=['Geodetic MB', 'MBM MB'])
ds_lstm_IS = ds_lstm_IS.sort_values(by='Area')
ds_lstm_IS['GLACIER'] = ds_lstm_IS['GLACIER'].apply(lambda x: x.capitalize())

### Mass balance gradients:

In [ ]:
# Stake data
# Load stake data ONCE instead of for every glacier
stake_file = os.path.join(cfg.dataPath, path_PMB_GLAMOS_csv,
                          "CH_wgms_dataset_all.csv")
df_stakes = pd.read_csv(stake_file)

#### Gradients test:

In [ ]:
def _alpha_labels(n: int):
    """(a), (b), ... (z), (aa), (ab), ... for n>=1"""

    def to_label(k: int) -> str:
        # 0 -> a, 25 -> z, 26 -> aa, ...
        s = ""
        k += 1
        while k > 0:
            k, r = divmod(k - 1, 26)
            s = chr(97 + r) + s
        return f"({s})"

    return [to_label(i) for i in range(n)]


In [ ]:
gl_list = [
    'Plattalva',
    'Hohlaub',
    'Tsanfleuron',
    'Schwarzberg',
    'Forno',
]

nrows = 1  # 0: OOS, 1: IS
ncols = len(gl_list)
cm = 1 / 2.54
fontsize = 6

fig, axs = plt.subplots(nrows=nrows,
                        ncols=ncols,
                        figsize=(28 * cm, 12 * cm),
                        dpi=300)

subplot_labels = _alpha_labels(ncols)
for c, gl in enumerate(gl_list):  # columns = glaciers
    # Annual
    df_lstm_a_oos, df_glamos_a_oos, df_all_a_oos = build_all_years_df(
        gl.lower(), PATH_PREDICTIONS_LSTM_OOS, cfg, period="annual")
    # Winter
    df_lstm_w_oos, df_glamos_w_oos, df_all_w_oos = build_all_years_df(
        gl.lower(), PATH_PREDICTIONS_LSTM_OOS, cfg, period="winter")

    # get unique years
    years = df_all_w_oos.YEAR.unique()

    ax = axs[c]

    # OOS: bands + mean + GLAMOS + stakes
    ax = plot_lstm_by_elevation_periods(df_all_a_oos,
                                        df_all_w_oos,
                                        ax=ax,
                                        mean_linestyle='-',
                                        label_prefix='LSTM OOS',
                                        show_band=True,
                                        color_annual=color_annual,
                                        color_winter=color_winter)

    ax = plot_glamos_by_elevation_periods(df_all_a_oos,
                                          df_all_w_oos,
                                          ax=ax,
                                          show_band=False,
                                          label_prefix="GLAMOS",
                                          mean_linestyle=":",
                                          color_annual=color_annual,
                                          color_winter=color_winter)

    # add stakes:
    ax = plot_stakes_by_elevation_periods(df_stakes,
                                          gl.lower(),
                                          valid_bins=None,
                                          ax=ax,
                                          color_annual=color_annual,
                                          color_winter=color_winter,
                                          marker_size=14)

    ax.set_ylabel('')
    ax.set_xlabel('')

    area = gl_area.get(gl.lower(), np.nan)
    area = np.round(area, 3) if area < 0.1 else np.round(area, 1)

    ax.set_title(f'{gl} ({area} km2, {years.min()}-{years.max()})',
                 fontsize=fontsize,
                 pad=2)

    # Row label on the left margin (first column only)
    if c == 0:
        ax.set_ylabel(f'Elevation (m a.s.l.)', fontsize=fontsize)

    ax.grid(alpha=0.2)
    ax.tick_params(labelsize=6.5, pad=2)
    ax.set_xlabel('')  # we use a supxlabel below

    # Subplot label (auto or provided)
    ax.text(
        0.02,
        0.98,
        subplot_labels[c],
        transform=ax.transAxes,
        fontsize=fontsize,
        va="top",
        ha="left",
    )

    # remove per-axes legend
    leg = ax.legend()
    if leg is not None:
        leg.remove()

    print(f'{gl}: {years.min()}-{years.max()}')

# Global x-label
fig.supxlabel('Mass balance (m w.e.)', fontsize=fontsize, y=0.06)

# Custom handles (bands, means, and stakes)
handles = [
    # LSTM
    Patch(facecolor=color_annual,
          alpha=0.25,
          label="MBM out-of-sample band (annual)"),
    Line2D([0], [0],
           color=color_annual,
           lw=1.2,
           linestyle='-',
           label="MBM out-of-sample mean (annual)"),
    Patch(facecolor=color_winter,
          alpha=0.25,
          label="MBM out-of-sample band (winter)"),
    Line2D([0], [0],
           color=color_winter,
           lw=1.2,
           linestyle='-',
           label="MBM out-of-sample mean (winter)"),

    # GLAMOS (mean only)
    Line2D([0], [0],
           color=color_annual,
           lw=1.2,
           linestyle=':',
           label="GLAMOS mean (annual)"),
    Line2D([0], [0],
           color=color_winter,
           lw=1.2,
           linestyle=':',
           label="GLAMOS mean (winter)"),

    # Stakes means
    Line2D([0], [0],
           marker='o',
           linestyle='None',
           linewidth=0,
           markersize=6,
           markerfacecolor='none',
           markeredgecolor=color_annual,
           markeredgewidth=1.2,
           label="Stakes mean (annual)"),
    Line2D([0], [0],
           marker='s',
           linestyle='None',
           linewidth=0,
           markersize=6,
           markerfacecolor='none',
           markeredgecolor=color_winter,
           markeredgewidth=1.2,
           label="Stakes mean (winter)"),
]

fig.legend(handles=handles,
           loc='upper center',
           bbox_to_anchor=(0.5, 0.05),
           ncol=4,
           fontsize=7)

plt.subplots_adjust(hspace=0.25, wspace=0.25)
plt.show()

# save figure
fig.savefig('figures/paper/fig_mb_gradients_testgl_OOS.png',
            dpi=300,
            bbox_inches='tight')

In [ ]:
test_gl_area = {}
for x in TEST_GLACIERS:
    test_gl_area[x] = gl_area[x]
test_gl_area = dict(
    sorted(test_gl_area.items(), key=lambda item: item[1], reverse=True))
test_gl_area

## In-sample results:

### Maps: Two glaciers, two years

In [ ]:
fig, map_axes = plot_2glaciers_2years_glamos_vs_lstm(
    glacier_names=("aletsch", "rhone"),
    years_by_glacier=((2014, 2022), (2009, 2022)),
    cfg=cfg,
    df_stakes=df_stakes,
    path_distributed_mb=path_distributed_MB_glamos,
    path_pred_lstm=PATH_PREDICTIONS_LSTM_IS,
    period="annual",
    panel_label_start="e",
    apply_smoothing_fn=apply_gaussian_filter,
)

# save figure
fig.savefig('figures/paper/fig_glamos_vs_lstm_aletsch_rhone.png',
            dpi=300,
            bbox_inches='tight')

### Gradients train:

In [ ]:
gl_list = [
    'Gries',
    'Gietro',
    'Rhone',
    'Aletsch',
]

nrows = 1  # 0: OOS, 1: IS
ncols = len(gl_list)
cm = 1 / 2.54
fontsize = 7

fig, axs = plt.subplots(nrows=nrows,
                        ncols=ncols,
                        figsize=(25 * cm, 12 * cm),
                        dpi=300)

subplot_labels = _alpha_labels(ncols)

for c, gl in enumerate(gl_list):  # columns = glaciers
    # Annual
    df_lstm_a_is, df_glamos_a_is, df_all_a_is = build_all_years_df(
        gl.lower(), PATH_PREDICTIONS_LSTM_IS, cfg, period="annual")
    # Winter
    df_lstm_w_is, df_glamos_w_is, df_all_w_is = build_all_years_df(
        gl.lower(), PATH_PREDICTIONS_LSTM_IS, cfg, period="winter")

    # get unique years
    years = df_all_w_oos.YEAR.unique()

    ax = axs[c]

    # IS: LSTM mean-only overlay (no band), dashed line to distinguish
    ax = plot_lstm_by_elevation_periods(df_all_a_is,
                                        df_all_w_is,
                                        ax=ax,
                                        mean_linestyle='-',
                                        label_prefix='LSTM IS',
                                        show_band=True,
                                        color_annual=color_annual,
                                        color_winter=color_winter)

    ax = plot_glamos_by_elevation_periods(df_all_a_is,
                                          df_all_w_is,
                                          ax=ax,
                                          show_band=False,
                                          label_prefix="GLAMOS",
                                          mean_linestyle=":",
                                          color_annual=color_annual,
                                          color_winter=color_winter)

    # add stakes:
    ax = plot_stakes_by_elevation_periods(df_stakes,
                                          gl.lower(),
                                          valid_bins=None,
                                          ax=ax,
                                          color_annual=color_annual,
                                          color_winter=color_winter,
                                          marker_size=14)

    ax.set_ylabel('')
    ax.set_xlabel('')

    area = gl_area.get(gl.lower(), np.nan)
    area = np.round(area, 3) if area < 0.1 else np.round(area, 1)

    ax.set_title(f'{gl} ({area} km2, {years.min()}-{years.max()})',
                 fontsize=fontsize,
                 pad=2)

    # Row label on the left margin (first column only)
    if c == 0:
        ax.set_ylabel(f'Elevation (m a.s.l.)', fontsize=fontsize)

    ax.grid(alpha=0.2)
    ax.tick_params(labelsize=6.5, pad=2)
    ax.set_xlabel('')  # we use a supxlabel below

    # Subplot label (auto or provided)
    ax.text(
        0.02,
        0.98,
        subplot_labels[c],
        transform=ax.transAxes,
        fontsize=fontsize,
        va="top",
        ha="left",
    )

    # remove per-axes legend
    leg = ax.legend()
    if leg is not None:
        leg.remove()

# Global x-label
fig.supxlabel('Mass balance (m w.e.)', fontsize=fontsize, y=0.06)

# Custom handles (bands, means, and stakes)
handles = [
    # LSTM
    Patch(facecolor=color_annual,
          alpha=0.25,
          label="MBM in-sample band (annual)"),
    Line2D([0], [0],
           color=color_annual,
           lw=1.2,
           linestyle='-',
           label="MBM in-sample mean (annual)"),
    Patch(facecolor=color_winter,
          alpha=0.25,
          label="MBM in-sample band (winter)"),
    Line2D([0], [0],
           color=color_winter,
           lw=1.2,
           linestyle='-',
           label="MBM in-sample mean (winter)"),

    # LSTM IS (mean only)
    Line2D([0], [0],
           color=color_annual,
           lw=1.2,
           linestyle=':',
           label="GLAMOS mean (annual)"),
    Line2D([0], [0],
           color=color_winter,
           lw=1.2,
           linestyle=':',
           label="GLAMOS mean (winter)"),

    # Stakes means
    Line2D([0], [0],
           marker='o',
           linestyle='None',
           linewidth=0,
           markersize=6,
           markerfacecolor='none',
           markeredgecolor=color_annual,
           markeredgewidth=1.2,
           label="Stakes mean (annual)"),
    Line2D([0], [0],
           marker='s',
           linestyle='None',
           linewidth=0,
           markersize=6,
           markerfacecolor='none',
           markeredgecolor=color_winter,
           markeredgewidth=1.2,
           label="Stakes mean (winter)"),
]

fig.legend(handles=handles,
           loc='upper center',
           bbox_to_anchor=(0.5, 0.05),
           ncol=5,
           fontsize=7)

plt.subplots_adjust(hspace=0.25, wspace=0.25)
plt.show()

# save figure
fig.savefig('figures/paper/fig_mb_gradients_IS.png',
            dpi=300,
            bbox_inches='tight')

### Combined: 

In [ ]:
import os
import numpy as np
import xarray as xr
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
from matplotlib.patches import Patch
from matplotlib.lines import Line2D
import string

# =====================================================
# --- Helpers ---
# =====================================================
def pick_file_glamos(glacier, year, period="annual"):
    suffix = "ann" if period == "annual" else "win"
    base = os.path.join(cfg.dataPath, path_distributed_MB_glamos, "GLAMOS", glacier)
    cand_lv95 = os.path.join(base, f"{year}_{suffix}_fix_lv95.grid")
    cand_lv03 = os.path.join(base, f"{year}_{suffix}_fix_lv03.grid")
    if os.path.exists(cand_lv95):
        return cand_lv95, "lv95"
    if os.path.exists(cand_lv03):
        return cand_lv03, "lv03"
    return None, None

def load_glamos_wgs84(glacier, year):
    path, cs = pick_file_glamos(glacier, year, period)
    if path is None:
        return None
    meta, arr = load_grid_file(path)
    da = convert_to_xarray_geodata(arr, meta)
    if cs == "lv03":
        return transform_xarray_coords_lv03_to_wgs84(da)
    if cs == "lv95":
        return transform_xarray_coords_lv95_to_wgs84(da)
    return None

def load_lstm_ds(glacier, year):
    zpath = os.path.join(PATH_PREDICTIONS_LSTM_IS, glacier, f"{glacier}_{year}_{period}.zarr")
    if not os.path.exists(zpath):
        return None
    ds = xr.open_zarr(zpath)
    ds = apply_gaussian_filter(ds)
    return ds

def lonlat_names(obj):
    coords = getattr(obj, "coords", {})
    if "lon" in coords and "lat" in coords:
        return "lon", "lat"
    if "longitude" in coords and "latitude" in coords:
        return "longitude", "latitude"
    return "lon", "lat"

def stake_overlay_rmse(ax, glacier, year, cmap, norm, da_glamos, ds_lstm, which):
    if df_stakes is None:
        return None
    sub = df_stakes[(df_stakes.GLACIER == glacier) & (df_stakes.YEAR == year)].copy()
    if period == "annual" and "PERIOD" in sub.columns:
        sub = sub[sub.PERIOD == "annual"].copy()
    if sub.empty:
        return None

    lx, ly = lonlat_names(ds_lstm if which == "LSTM" and ds_lstm is not None else da_glamos)

    def _safe_pred(ds, row):
        try:
            return get_predicted_mb(lx, ly, row, ds)
        except Exception:
            return np.nan

    def _safe_glamos(row):
        try:
            return get_predicted_mb_glamos(lx, ly, row, da_glamos)
        except Exception:
            return np.nan

    if which == "GLAMOS":
        sub["FIELD"] = sub.apply(_safe_glamos, axis=1)
    else:
        sub["FIELD"] = (
            sub.apply(lambda r: _safe_pred(ds_lstm, r), axis=1)
            if ds_lstm is not None
            else np.nan
        )

    hue_col = "POINT_BALANCE" if "POINT_BALANCE" in sub.columns else "FIELD"
    sns.scatterplot(
        data=sub,
        x="POINT_LON",
        y="POINT_LAT",
        hue=hue_col,
        palette=cmap,
        hue_norm=norm,
        ax=ax,
        s=18,
        legend=False,
    )

    if "POINT_BALANCE" in sub.columns and not np.all(np.isnan(sub["FIELD"])):
        return root_mean_squared_error(sub["POINT_BALANCE"], sub["FIELD"])
    return None


# =====================================================
# --- Configuration ---
# =====================================================
glacier_names = ("aletsch", "rhone")
years_by_glacier = ((2014, 2022), (2009, 2022))
grad_glaciers = ["Gries", "Gietro", "Rhone", "Aletsch"]
period = "annual"
fontsize = 7
cm = 1 / 2.54

# =====================================================
# --- Figure setup (3 rows total) ---
# =====================================================
fig = plt.figure(figsize=(25 * cm, 30 * cm), dpi=300)

# Top two rows for maps (2×6 grid), third row reserved for gradients (1×4)
gs_main = GridSpec(
    3, 6, figure=fig,
    width_ratios=[1, 1, 0.045, 1, 1, 0.045],
    height_ratios=[1, 1, 1],
    wspace=0.3, hspace=0.25
)
gs_grad = gs_main[2, :].subgridspec(
    1, 6,
    width_ratios=[1, 1, 0.045, 1, 1, 0.045],
    wspace=0.3
)
label_iter = iter(string.ascii_lowercase)

# =====================================================
# --- ROWS 1–2: GLAMOS vs MBM maps ---
# =====================================================
for r, glacier in enumerate(glacier_names):
    row_years = years_by_glacier[r]
    for j, year in enumerate(row_years):
        col_base = 3 * j
        ax_g = fig.add_subplot(gs_main[r, col_base + 0])
        ax_m = fig.add_subplot(gs_main[r, col_base + 1])
        ax_cb = fig.add_subplot(gs_main[r, col_base + 2])

        da_g = load_glamos_wgs84(glacier, year)
        ds_m = load_lstm_ds(glacier, year)

        vals = []
        if da_g is not None:
            vals += [float(da_g.min().item()), float(da_g.max().item())]
        if ds_m is not None and "pred_masked" in ds_m:
            vals += [float(ds_m["pred_masked"].min().item()), float(ds_m["pred_masked"].max().item())]

        if not vals:
            for ax in (ax_g, ax_m):
                ax.text(0.5, 0.5, f"No data\n{glacier} {year}", ha="center", va="center")
                ax.set_axis_off()
            ax_cb.set_axis_off()
            continue

        vmin, vmax = min(vals), max(vals)
        cmap, norm, _, _ = get_color_maps(vmin, vmax, 0, 0)

        # --- GLAMOS ---
        if da_g is not None:
            da_g.plot.imshow(ax=ax_g, cmap=cmap, norm=norm, add_colorbar=False)
            ax_g.set_title(f"{glacier.capitalize()} – GLAMOS ({year})", fontsize=9)
            mean_g = float(da_g.mean())
            var_g = float(da_g.var())
            rmse_g = stake_overlay_rmse(ax_g, glacier, year, cmap, norm, da_g, ds_m, "GLAMOS")
            txt = (f"RMSE: {rmse_g:.2f}\n" if rmse_g else "") + f"mean MB: {mean_g:.2f}\nvar: {var_g:.2f}"
            ax_g.text(0.04, 0.03, txt, transform=ax_g.transAxes, ha="left", va="bottom",
                      fontsize=6, bbox=dict(facecolor="white", alpha=0.7, edgecolor="none"))


        # --- LSTM ---
        if ds_m is not None and "pred_masked" in ds_m:
            ds_m["pred_masked"].plot.imshow(ax=ax_m, cmap=cmap, norm=norm, add_colorbar=False)
            ax_m.set_title(f"{glacier.capitalize()} – MBM ({year})", fontsize=9)
            mean_m = float(ds_m["pred_masked"].mean())
            var_m = float(ds_m["pred_masked"].var())
            rmse_m = stake_overlay_rmse(ax_m, glacier, year, cmap, norm, da_g, ds_m, "LSTM")
            txt = (f"RMSE: {rmse_m:.2f}\n" if rmse_m else "") + f"mean MB: {mean_m:.2f}\nvar: {var_m:.2f}"
            ax_m.text(0.04, 0.03, txt, transform=ax_m.transAxes, ha="left", va="bottom",
                      fontsize=6, bbox=dict(facecolor="white", alpha=0.7, edgecolor="none"))

        # --- Colorbar ---
        mappable = ds_m["pred_masked"] if ds_m is not None else da_g
        cb = fig.colorbar(mappable.plot.imshow(cmap=cmap, norm=norm, add_colorbar=False), cax=ax_cb)
        if j == 5:
            cb.set_label("Mass Balance [m w.e.]", fontsize=7)
        else:
            cb.set_label("")
        cb.ax.tick_params(labelsize=6)
        ax_cb.set_xlabel("")
        

        # --- Labels ---
        ax_g.set_ylabel("Latitude" if j == 0 else "", fontsize = 7)
        ax_m.set_ylabel("")
        ax_g.set_xlabel("Longitude" if r == 1 else "", fontsize = 7)
        ax_m.set_xlabel("Longitude" if r == 1 else "", fontsize = 7)
        if j > 0:
            ax_g.tick_params(labelleft=False,labelsize = 7)
        else:
            ax_g.tick_params(labelsize = 7)
        ax_m.tick_params(labelleft=False,labelsize = 7)

        # --- Panel labels ---
        for ax in (ax_g, ax_m):
            label = next(label_iter)
            ax.text(0.02, 0.98, f"({label})", transform=ax.transAxes,
                    ha="left", va="top", fontsize=8,
                    bbox=dict(facecolor="white", alpha=0.5, edgecolor="none"))


# =====================================================
# --- ROW 3: Gradients (aligned to 4 main columns) ---
# =====================================================
grad_col_indices = [0, 1, 3, 4]  # skip colorbar columns

for c, gl in enumerate(grad_glaciers):
    ax = fig.add_subplot(gs_grad[0, grad_col_indices[c]])
    df_lstm_a_is, df_glamos_a_is, df_all_a_is = build_all_years_df(
        gl.lower(), PATH_PREDICTIONS_LSTM_IS, cfg, "annual")
    df_lstm_w_is, df_glamos_w_is, df_all_w_is = build_all_years_df(
        gl.lower(), PATH_PREDICTIONS_LSTM_IS, cfg, "winter")

    ax = plot_lstm_by_elevation_periods(df_all_a_is, df_all_w_is, ax=ax,
                                        mean_linestyle='-',
                                        label_prefix='MBM IS', show_band=True,
                                        color_annual=color_annual, color_winter=color_winter)
    ax = plot_glamos_by_elevation_periods(df_all_a_is, df_all_w_is, ax=ax,
                                          show_band=False,
                                          label_prefix="GLAMOS",
                                          mean_linestyle=":",
                                          color_annual=color_annual, color_winter=color_winter)
    ax = plot_stakes_by_elevation_periods(df_stakes, gl.lower(),
                                          valid_bins=None, ax=ax,
                                          color_annual=color_annual, color_winter=color_winter,
                                          marker_size=14)

    #ax.set_title(gl, fontsize=7)
    area = gl_area.get(gl.lower(), np.nan)
    area = np.round(area, 3) if area < 0.1 else np.round(area, 1)

    ax.set_title(f'{gl} ({area} km2, {years.min()}-{years.max()})',
                 fontsize=fontsize,
                 pad=2)
    ax.grid(alpha=0.2)
    ax.tick_params(labelsize=6, pad=2)
    ax.text(0.02, 0.98, f"({next(label_iter)})", transform=ax.transAxes,
            ha="left", va="top", fontsize=8,
            bbox=dict(facecolor="white", alpha=0.5, edgecolor="none"))

plt.tight_layout()
plt.show()

# Save
fig.savefig("figures/paper/fig_combined_glamos_mbm_gradients.png", dpi=300, bbox_inches="tight")

### Geodetic MB:

In [ ]:
# Compute RMSE and Pearson correlation
rmse_nn = root_mean_squared_error(ds_lstm_OS["Geodetic MB"],
                                  ds_lstm_OS["MBM MB"])
corr_nn = np.corrcoef(ds_lstm_OS["Geodetic MB"], ds_lstm_OS["MBM MB"])[0, 1]

fig = plot_mbm_vs_geodetic_by_area_bin(
    ds_lstm_OS,
    bins=[0, 1, 5, 10, 100, np.inf],
    labels=['<1', '1-5', '5–10', '>10', '>100'],
    max_bins=4)

# save figure
fig.savefig('figures/paper/fig_mbm_vs_geodetic_by_area_bin_OOS.png',
            dpi=300,
            bbox_inches='tight')

In [ ]:
# Compute RMSE and Pearson correlation
rmse_nn = root_mean_squared_error(ds_lstm_IS["Geodetic MB"],
                                  ds_lstm_IS["MBM MB"])
corr_nn = np.corrcoef(ds_lstm_IS["Geodetic MB"], ds_lstm_IS["MBM MB"])[0, 1]

fig = plot_mbm_vs_geodetic_by_area_bin(
    ds_lstm_IS,
    bins=[0, 1, 5, 10, 100, np.inf],
    labels=['<1', '1-5', '5–10', '>10', '>100'],
    max_bins=4)

# save figure
fig.savefig('figures/paper/fig_mbm_vs_geodetic_by_area_bin_IS.png',
            dpi=300,
            bbox_inches='tight')

## Rest figures:

### Gradients all:

In [ ]:
nrows = 4
ncols = 5
cm = 1 / 2.54
fontsize = 7

# Create a figure with the specified number of subplots
fig, axs = plt.subplots(nrows=nrows,
                        ncols=ncols,
                        figsize=(25 * cm, 15 * cm),
                        dpi=300)
axs = axs.flatten()
gl_list = [
    'schwarzbach', 'murtel', 'plattalva', 'basodino', 'limmern', 'adler',
    'hohlaub', 'albigna', 'tsanfleuron', 'silvretta', 'gries', 'clariden',
    'gietro', 'schwarzberg', 'forno', 'allalin', 'otemma', 'findelen', 'rhone',
    'aletsch'
]
for i, gl in enumerate(gl_list):
    # Annual
    df_lstm_a, df_glamos_a, df_all_a = build_all_years_df(
        gl.lower(), PATH_PREDICTIONS_LSTM_IS, cfg, period="annual")

    years = df_all_a.YEAR.unique()

    # Winter
    df_lstm_w, df_glamos_w, df_all_w = build_all_years_df(
        gl.lower(), PATH_PREDICTIONS_LSTM_IS, cfg, period="winter")

    # if dataframe not None
    if df_all_a.empty:
        print(f"No data for glacier: {gl}")
        continue

    ax = plot_mb_by_elevation_periods_combined(df_all_a,
                                               df_all_w,
                                               df_stakes,
                                               gl.lower(),
                                               ax=axs[i])

    # area = areas_per_gl.loc[gl].Area
    area = gl_area.get(gl.lower(), np.nan)
    if area < 0.1:
        area = np.round(area, 3)
    else:
        area = np.round(area, 1)
    if gl.lower() in TEST_GLACIERS:
        axs[i].set_title(f'*{gl} ({area} km2, {years.min()}-{years.max()})',
                         fontsize=fontsize,
                         pad=2)
    else:
        axs[i].set_title(f'{gl} ({area} km2, {years.min()}-{years.max()})',
                         fontsize=fontsize,
                         pad=2)

    axs[i].grid(alpha=0.2)
    axs[i].tick_params(labelsize=6.5, pad=2)
    axs[i].set_ylabel('')
    axs[i].set_xlabel('')
    # remove legend
    axs[i].legend().remove()

axs[5].set_ylabel('Elevation (m a.s.l.)', fontsize=fontsize)

# Custom handles (bands, means, and stakes)
handles = [
    # LSTM
    Patch(facecolor=color_annual, alpha=0.25, label="LSTM band (annual)"),
    Line2D([0], [0],
           color=color_annual,
           lw=1.2,
           linestyle='-',
           label="LSTM mean (annual)"),
    Patch(facecolor=color_winter, alpha=0.25, label="LSTM band (winter)"),
    Line2D([0], [0],
           color=color_winter,
           lw=1.2,
           linestyle='-',
           label="LSTM mean (winter)"),

    # GLAMOS (mean only)
    Line2D([0], [0],
           color=color_annual,
           lw=1.2,
           linestyle=':',
           label="GLAMOS mean (annual)"),
    Line2D([0], [0],
           color=color_winter,
           lw=1.2,
           linestyle=':',
           label="GLAMOS mean (winter)"),

    # Stakes means
    Line2D([0], [0],
           marker='o',
           linestyle='None',
           linewidth=0,
           markersize=6,
           markerfacecolor='none',
           markeredgecolor=color_annual,
           markeredgewidth=1.2,
           label="Stakes mean (annual)"),
    Line2D([0], [0],
           marker='s',
           linestyle='None',
           linewidth=0,
           markersize=6,
           markerfacecolor='none',
           markeredgecolor=color_winter,
           markeredgewidth=1.2,
           label="Stakes mean (winter)"),
]

fig.supxlabel('Mass balance (m w.e.)', fontsize=fontsize, y=0.06)

fig.legend(handles=handles,
           loc='upper center',
           bbox_to_anchor=(0.5, 0.05),
           ncol=4,
           fontsize=7)

# Adjust the layout
plt.subplots_adjust(hspace=0.25, wspace=0.25)
plt.show()

### Maps:

In [ ]:
# Example usage
GLACIER_NAME = 'schwarzberg'
df_lstm_two_heads_gl = ds_lstm_IS[ds_lstm_IS.GLACIER ==
                                  GLACIER_NAME.capitalize()]

fig, axs = plt.subplots(1, 2, figsize=(15, 5))

plot_scatter_comparison(axs[0],
                        df_lstm_two_heads_gl,
                        GLACIER_NAME,
                        color_mbm=color_annual,
                        color_glamos=color_winter,
                        title_suffix="(LSTM two heads)")

# Load GLAMOS data
GLAMOS_glwmb = get_GLAMOS_glwmb(GLACIER_NAME, cfg)

MBM_glwmb_lstm = mbm_glwd_pred(PATH_PREDICTIONS_LSTM_IS, GLACIER_NAME)
MBM_glwmb_lstm.rename(columns={"MBM Balance": "MBM Balance LSTM"},
                      inplace=True)

# Merge with GLAMOS data
MBM_glwmb_lstm = MBM_glwmb_lstm.join(GLAMOS_glwmb)

# Plot the data
MBM_glwmb_lstm.plot(ax=axs[1],
                    y=['MBM Balance LSTM', 'GLAMOS Balance'],
                    marker="o",
                    color=[color_annual, color_winter])

axs[1].set_title(f"{GLACIER_NAME.capitalize()} Glacier", fontsize=24)
axs[1].set_ylabel("Mass Balance [m w.e.]", fontsize=18)
axs[1].set_xlabel("Year", fontsize=18)
axs[1].grid(True, linestyle="--", linewidth=0.5)
axs[1].legend(fontsize=14)
axs[1].set_title(f"{GLACIER_NAME.capitalize()} Glacier (LSTM)", fontsize=16)

plt.tight_layout()
plt.show()

In [ ]:
print(MBM_glwmb_lstm.index)
year_range = range(1956, 1960)
for year in year_range:
    plot_mass_balance_comparison_annual(
        glacier_name=GLACIER_NAME,
        year=year,
        cfg=cfg,
        df_stakes=df_stakes,
        path_distributed_mb=path_distributed_MB_glamos,
        path_pred_lstm=PATH_PREDICTIONS_LSTM_IS,
        period='annual')